In [1]:
import xmltodict
import pandas as pd
import os
import tarfile
import shutil

In [10]:
def get_version(doc):    
    # VERSION
    if '@VERSION' in doc.keys():
        return doc['@VERSION']
    else:
        if isinstance(doc['FORM_SECTION'][list(doc['FORM_SECTION'].keys())[0]], dict):
            return doc['FORM_SECTION'][list(doc['FORM_SECTION'].keys())[0]]['@VERSION']
        else:
            return doc['FORM_SECTION'][list(doc['FORM_SECTION'].keys())[0]][0]['@VERSION']
            
def get_lang_pos(doc):
    # EN_POS
    if 'EN' in doc['TECHNICAL_SECTION']['FORM_LG_LIST']:
        return doc['TECHNICAL_SECTION']['FORM_LG_LIST'].split(' ').index('EN')
    else:
        return 0
    
def get_form(doc):
    
    version = get_version(doc)
    EN_POS = get_lang_pos(doc)
    
    if version[:6] == 'R2.0.9':
        if 'F01_2014' in doc['FORM_SECTION'].keys():
            #todo: parse R2.0.9 'Prior Information Notice'
            return 'R2.0.9_PIN'
        
        elif 'F02_2014' in doc['FORM_SECTION'].keys():
            return R209_CN(doc, EN_POS)
        
        elif 'F03_2014' in doc['FORM_SECTION'].keys():
            #todo: parse R2.0.9 'Contract Award Notice'
            return 'R2.0.9_CAN'
        else:
            #pass
            return 'R2.0.9_' + list(doc['FORM_SECTION'].keys())[0]
    elif version[:6] == 'R2.0.8':
        if 'PRIOR_INFORMATION' in doc['FORM_SECTION'].keys():
            #todo: parse R2.0.8 'Prior Information Notice'
            return 'R2.0.8_PIN'
        elif 'CONTRACT' in doc['FORM_SECTION'].keys():
            #todo: parse R2.0.8 'Contract Notice'
            return 'R2.0.8_CN'
        elif 'CONTRACT_AWARD' in doc['FORM_SECTION'].keys():
            #todo: parse R2.0.8 'Contract Award Notice'
            return 'R2.0.8_CAN'
        else:
            #pass
            return 'R2.0.8_' + list(doc['FORM_SECTION'].keys())[0]

In [2]:
def R209_CN(doc, EN_POS):
    d = dict()
    if isinstance(doc['FORM_SECTION']['F02_2014'], list):
        d['FORM'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['@FORM']
        d['LEGAL_BASIS'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['LEGAL_BASIS']['@VALUE']
        d['CONTRACTING_BODY'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['CONTRACTING_BODY']['ADDRESS_CONTRACTING_BODY']['OFFICIALNAME']
        d['CONTRACTING_BODY_ADDRESS'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['CONTRACTING_BODY']['ADDRESS_CONTRACTING_BODY']['ADDRESS'] if 'ADDRESS' in doc['FORM_SECTION']['F02_2014'][EN_POS]['CONTRACTING_BODY']['ADDRESS_CONTRACTING_BODY'].keys() else ''
        d['CONTRACTING_BODY_TOWN'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['CONTRACTING_BODY']['ADDRESS_CONTRACTING_BODY']['TOWN'] if 'TOWN' in doc['FORM_SECTION']['F02_2014'][EN_POS]['CONTRACTING_BODY']['ADDRESS_CONTRACTING_BODY'].keys() else ''
        d['CONTRACTING_BODY_POSTAL_CODE'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['CONTRACTING_BODY']['ADDRESS_CONTRACTING_BODY']['POSTAL_CODE'] if 'POSTAL_CODE' in doc['FORM_SECTION']['F02_2014'][EN_POS]['CONTRACTING_BODY']['ADDRESS_CONTRACTING_BODY'].keys() else ''
        d['CONTRACTING_BODY_COUNTRY'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['CONTRACTING_BODY']['ADDRESS_CONTRACTING_BODY']['COUNTRY']['@VALUE'] if 'COUNTRY' in doc['FORM_SECTION']['F02_2014'][EN_POS]['CONTRACTING_BODY']['ADDRESS_CONTRACTING_BODY'].keys() else ''
        d['n2016-NUTS'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['CONTRACTING_BODY']['ADDRESS_CONTRACTING_BODY']['n2016:NUTS']['@CODE']
        d['CA_TYPE'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['CONTRACTING_BODY']['CA_TYPE']['@VALUE'] if 'CA_TYPE' in doc['FORM_SECTION']['F02_2014'][EN_POS]['CONTRACTING_BODY'].keys() else ''
        d['CA_TYPE_OTHER'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['CONTRACTING_BODY']['CA_TYPE_OTHER'] if 'CA_TYPE_OTHER' in doc['FORM_SECTION']['F02_2014'][EN_POS]['CONTRACTING_BODY'].keys() else ''
        d['CA_ACTIVITY'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['CONTRACTING_BODY']['CA_ACTIVITY']['@VALUE'] if 'CA_ACTIVITY' in doc['FORM_SECTION']['F02_2014'][EN_POS]['CONTRACTING_BODY'].keys() else ''
        d['CA_ACTIVITY_OTHER'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['CONTRACTING_BODY']['CA_ACTIVITY_OTHER'] if 'CA_ACTIVITY_OTHER' in doc['FORM_SECTION']['F02_2014'][EN_POS]['CONTRACTING_BODY'].keys() else ''
        d['TITLE'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['TITLE']['P']
        d['REFERENCE_NUMBER'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['REFERENCE_NUMBER'] if 'REFERENCE_NUMBER' in doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT'].keys() else ''
        d['CPV_MAIN'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['CPV_MAIN']['CPV_CODE']['@CODE']
        d['TYPE_CONTRACT'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['TYPE_CONTRACT']['@CTYPE']
        try:
            d['SHORT_DESCR'] = ''.join(doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['SHORT_DESCR']['P'])
        except:
            d['SHORT_DESCR'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['SHORT_DESCR']['P']
        if 'NO_LOT_DIVISION' in doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT'].keys():
            d['NO_LOT_DIVISION'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['NO_LOT_DIVISION']
        if 'LOT_DIVISION' in doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT'].keys():
            d['MAX_LOT_DIVISION'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['LOT_DIVISION']['LOT_MAX_NUMBER'] if 'LOT_MAX_NUMBER' in doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['LOT_DIVISION'].keys() else ''
            d['MAX_LOT_PER_TENDERER'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['LOT_DIVISION']['LOT_MAX_ONE_TENDERER'] if 'LOT_MAX_ONE_TENDERER' in doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['LOT_DIVISION'].keys() else ''
            d['LOT_ALL'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['LOT_DIVISION']['LOT_ALL'] if 'LOT_ALL' in doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['LOT_DIVISION'].keys() else ''
        if isinstance(doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['OBJECT_DESCR'], dict):
            if 'CPV_ADDITIONAL' in doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['OBJECT_DESCR'].keys():
                if isinstance(doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['OBJECT_DESCR']['CPV_ADDITIONAL'], dict):
                    d['OBJECT_DESCR_CPV_ADDITIONAL'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['OBJECT_DESCR']['CPV_ADDITIONAL']['CPV_CODE']['@CODE'] if 'CPV_ADDITIONAL' in doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['OBJECT_DESCR'].keys() else ''
                else:
                    d['OBJECT_DESCR_CPV_ADDITIONAL'] = ','.join([item['CPV_CODE']['@CODE'] for item in doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['OBJECT_DESCR']['CPV_ADDITIONAL']]) 
            if isinstance(doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['OBJECT_DESCR']['n2016:NUTS'], dict):
                d['OBJECT_DESCR_n2016-NUTS'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['OBJECT_DESCR']['n2016:NUTS']['@CODE']
            else:
                d['OBJECT_DESCR_n2016-NUTS'] = ','.join(item['@CODE'] for item in doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['OBJECT_DESCR']['n2016:NUTS'])
            d['OBJECT_DESCR_SHORT_DESCR'] = ''.join([s['#text'] if isinstance(s, dict) else s for s in doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['OBJECT_DESCR']['SHORT_DESCR']['P']])
            if 'DURATION' in doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['OBJECT_DESCR'].keys():
                d['OBJECT_DESCR_DURATION'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['OBJECT_DESCR']['DURATION']['#text'] + ' ' + doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['OBJECT_DESCR']['DURATION']['@TYPE']
        elif isinstance(doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['OBJECT_DESCR'], list):
            if 'CPV_ADDITIONAL' in doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['OBJECT_DESCR'].keys():
                if isinstance(doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['OBJECT_DESCR']['CPV_ADDITIONAL'], dict):
                    d['OBJECT_DESCR_CPV_ADDITIONAL'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['OBJECT_DESCR'][EN_POS]['CPV_ADDITIONAL']['CPV_CODE']['@CODE'] if 'CPV_ADDITIONAL' in doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['OBJECT_DESCR'][EN_POS].keys() else ''
                else:
                    d['OBJECT_DESCR_CPV_ADDITIONAL'] = ','.join([item['CPV_CODE']['@CODE'] for item in doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['OBJECT_DESCR'][EN_POS]['CPV_ADDITIONAL']])
            if isinstance(doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['OBJECT_DESCR']['n2016:NUTS'], dict):
                d['OBJECT_DESCR_n2016-NUTS'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['OBJECT_DESCR']['n2016:NUTS']['@CODE']
            else:
                d['OBJECT_DESCR_n2016-NUTS'] = ','.join(item['@CODE'] for item in doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['OBJECT_DESCR']['n2016:NUTS'])
            d['OBJECT_DESCR_SHORT_DESCR'] = ''.join([s['#text'] if isinstance(s, dict) else s for s in doc['FORM_SECTION'][EN_POS]['F02_2014']['OBJECT_CONTRACT']['OBJECT_DESCR'][EN_POS]['SHORT_DESCR']['P']])
            if 'DURATION' in doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['OBJECT_DESCR'][EN_POS].keys():
                d['OBJECT_DESCR_DURATION'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['OBJECT_DESCR'][EN_POS]['DURATION']['#text'] + ' ' + doc['FORM_SECTION']['F02_2014'][EN_POS]['OBJECT_CONTRACT']['OBJECT_DESCR'][EN_POS]['DURATION']['@TYPE']
        d['PROCEDURE_DATE_RECEIPT_TENDERS'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['PROCEDURE']['DATE_RECEIPT_TENDERS'] if 'DATE_RECEIPT_TENDERS' in doc['FORM_SECTION']['F02_2014'][EN_POS]['PROCEDURE'].keys() else ''
        d['PROCEDURE_TIME_RECEIPT_TENDERS'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['PROCEDURE']['TIME_RECEIPT_TENDERS'] if 'TIME_RECEIPT_TENDERS' in doc['FORM_SECTION']['F02_2014'][EN_POS]['PROCEDURE'].keys() else ''
        d['PROCEDURE_DURATION_TENDER_VALID'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['PROCEDURE']['DURATION_TENDER_VALID']['#text'] + ' ' + doc['FORM_SECTION']['F02_2014'][EN_POS]['PROCEDURE']['DURATION_TENDER_VALID']['@TYPE'] if 'DURATION_TENDER_VALID' in doc['FORM_SECTION']['F02_2014'][EN_POS]['PROCEDURE'].keys() else ''
        if 'OPENING_CONDITION' in doc['FORM_SECTION']['F02_2014'][EN_POS]['PROCEDURE'].keys():
            d['PROCEDURE_DATE_OPENING_TENDERS'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['PROCEDURE']['OPENING_CONDITION']['DATE_OPENING_TENDERS'] if 'DATE_OPENING_TENDERS' in doc['FORM_SECTION']['F02_2014'][EN_POS]['PROCEDURE']['OPENING_CONDITION'].keys() else ''
            d['PROCEDURE_TIME_OPENING_TENDERS'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['PROCEDURE']['OPENING_CONDITION']['TIME_OPENING_TENDERS'] if 'TIME_OPENING_TENDERS' in doc['FORM_SECTION']['F02_2014'][EN_POS]['PROCEDURE']['OPENING_CONDITION'].keys() else ''
        d['DATE_DISPATCH_NOTICE'] = doc['FORM_SECTION']['F02_2014'][EN_POS]['COMPLEMENTARY_INFO']['DATE_DISPATCH_NOTICE'] if 'DATE_DISPATCH_NOTICE' in doc['FORM_SECTION']['F02_2014'][EN_POS]['COMPLEMENTARY_INFO'].keys() else ''
    
    elif isinstance(doc['FORM_SECTION']['F02_2014'], dict):
        d['FORM'] = doc['FORM_SECTION']['F02_2014']['@FORM']
        d['LEGAL_BASIS'] = doc['FORM_SECTION']['F02_2014']['LEGAL_BASIS']['@VALUE']
        d['CONTRACTING_BODY'] = doc['FORM_SECTION']['F02_2014']['CONTRACTING_BODY']['ADDRESS_CONTRACTING_BODY']['OFFICIALNAME']
        d['CONTRACTING_BODY_ADDRESS'] = doc['FORM_SECTION']['F02_2014']['CONTRACTING_BODY']['ADDRESS_CONTRACTING_BODY']['ADDRESS'] if 'ADDRESS' in doc['FORM_SECTION']['F02_2014']['CONTRACTING_BODY']['ADDRESS_CONTRACTING_BODY'].keys() else ''
        d['CONTRACTING_BODY_TOWN'] = doc['FORM_SECTION']['F02_2014']['CONTRACTING_BODY']['ADDRESS_CONTRACTING_BODY']['TOWN'] if 'TOWN' in doc['FORM_SECTION']['F02_2014']['CONTRACTING_BODY']['ADDRESS_CONTRACTING_BODY'].keys() else ''
        d['CONTRACTING_BODY_POSTAL_CODE'] = doc['FORM_SECTION']['F02_2014']['CONTRACTING_BODY']['ADDRESS_CONTRACTING_BODY']['POSTAL_CODE'] if 'POSTAL_CODE' in doc['FORM_SECTION']['F02_2014']['CONTRACTING_BODY']['ADDRESS_CONTRACTING_BODY'].keys() else ''
        d['CONTRACTING_BODY_COUNTRY'] = doc['FORM_SECTION']['F02_2014']['CONTRACTING_BODY']['ADDRESS_CONTRACTING_BODY']['COUNTRY']['@VALUE'] if 'COUNTRY' in doc['FORM_SECTION']['F02_2014']['CONTRACTING_BODY']['ADDRESS_CONTRACTING_BODY'].keys() else ''
        d['n2016-NUTS'] = doc['FORM_SECTION']['F02_2014']['CONTRACTING_BODY']['ADDRESS_CONTRACTING_BODY']['n2016:NUTS']['@CODE']
        d['CA_TYPE'] = doc['FORM_SECTION']['F02_2014']['CONTRACTING_BODY']['CA_TYPE']['@VALUE'] if 'CA_TYPE' in doc['FORM_SECTION']['F02_2014']['CONTRACTING_BODY'].keys() else ''
        d['CA_TYPE_OTHER'] = doc['FORM_SECTION']['F02_2014']['CONTRACTING_BODY']['CA_TYPE_OTHER'] if 'CA_TYPE_OTHER' in doc['FORM_SECTION']['F02_2014']['CONTRACTING_BODY'].keys() else ''
        d['CA_ACTIVITY'] = doc['FORM_SECTION']['F02_2014']['CONTRACTING_BODY']['CA_ACTIVITY']['@VALUE'] if 'CA_ACTIVITY' in doc['FORM_SECTION']['F02_2014']['CONTRACTING_BODY'].keys() else ''
        d['CA_ACTIVITY_OTHER'] = doc['FORM_SECTION']['F02_2014']['CONTRACTING_BODY']['CA_ACTIVITY_OTHER'] if 'CA_ACTIVITY_OTHER' in doc['FORM_SECTION']['F02_2014']['CONTRACTING_BODY'].keys() else ''
        d['TITLE'] = doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['TITLE']['P']
        d['REFERENCE_NUMBER'] = doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['REFERENCE_NUMBER'] if 'REFERENCE_NUMBER' in doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT'].keys() else ''
        d['CPV_MAIN'] = doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['CPV_MAIN']['CPV_CODE']['@CODE']
        d['TYPE_CONTRACT'] = doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['TYPE_CONTRACT']['@CTYPE']
        try:
            d['SHORT_DESCR'] = ''.join(doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['SHORT_DESCR']['P'])
        except:
            d['SHORT_DESCR'] = doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['SHORT_DESCR']['P']

        if 'NO_LOT_DIVISION' in doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT'].keys():
            d['NO_LOT_DIVISION'] = doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['NO_LOT_DIVISION']
        if 'LOT_DIVISION' in doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT'].keys():
            d['MAX_LOT_DIVISION'] = doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['LOT_DIVISION']['LOT_MAX_NUMBER'] if 'LOT_MAX_NUMBER' in doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['LOT_DIVISION'].keys() else ''
            d['MAX_LOT_PER_TENDERER'] = doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['LOT_DIVISION']['LOT_MAX_ONE_TENDERER'] if 'LOT_MAX_ONE_TENDERER' in doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['LOT_DIVISION'].keys() else ''
            d['LOT_ALL'] = doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['LOT_DIVISION']['LOT_ALL'] if 'LOT_ALL' in doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['LOT_DIVISION'].keys() else ''
        if isinstance(doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['OBJECT_DESCR'], dict):
            if 'CPV_ADDITIONAL' in doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['OBJECT_DESCR'].keys():
                if isinstance(doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['OBJECT_DESCR']['CPV_ADDITIONAL'], dict):
                    d['OBJECT_DESCR_CPV_ADDITIONAL'] = doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['OBJECT_DESCR']['CPV_ADDITIONAL']['CPV_CODE']['@CODE']
                else:
                    d['OBJECT_DESCR_CPV_ADDITIONAL'] = ','.join([item['CPV_CODE']['@CODE'] for item in doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['OBJECT_DESCR']['CPV_ADDITIONAL']])
            if isinstance(doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['OBJECT_DESCR']['n2016:NUTS'], dict):
                d['OBJECT_DESCR_n2016-NUTS'] = doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['OBJECT_DESCR']['n2016:NUTS']['@CODE']
            else:
                d['OBJECT_DESCR_n2016-NUTS'] = ','.join(item['@CODE'] for item in doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['OBJECT_DESCR']['n2016:NUTS'])
            d['OBJECT_DESCR_SHORT_DESCR'] = ''.join([s['#text'] if isinstance(s, dict) else s for s in doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['OBJECT_DESCR']['SHORT_DESCR']['P']])
            if 'DURATION' in doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['OBJECT_DESCR'].keys():
                d['OBJECT_DESCR_DURATION'] = doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['OBJECT_DESCR']['DURATION']['#text'] + ' ' + doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['OBJECT_DESCR']['DURATION']['@TYPE']
        elif isinstance(doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['OBJECT_DESCR'], list):
            if 'CPV_ADDITIONAL' in doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['OBJECT_DESCR'][0].keys():
                if isinstance(doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['OBJECT_DESCR'][0]['CPV_ADDITIONAL'], dict):
                    d['OBJECT_DESCR_CPV_ADDITIONAL'] = doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['OBJECT_DESCR'][0]['CPV_ADDITIONAL']['CPV_CODE']['@CODE']
                else:
                    d['OBJECT_DESCR_CPV_ADDITIONAL'] = ','.join([item['CPV_CODE']['@CODE'] for item in doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['OBJECT_DESCR'][0]['CPV_ADDITIONAL']])
            if isinstance(doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['OBJECT_DESCR'][0]['n2016:NUTS'], dict):
                d['OBJECT_DESCR_n2016-NUTS'] = doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['OBJECT_DESCR'][0]['n2016:NUTS']['@CODE']
            else:
                d['OBJECT_DESCR_n2016-NUTS'] = ','.join(item['@CODE'] for item in doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['OBJECT_DESCR'][0]['n2016:NUTS'])
            d['OBJECT_DESCR_SHORT_DESCR'] = ''.join([s['#text'] if isinstance(s, dict) else s for s in doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['OBJECT_DESCR'][0]['SHORT_DESCR']['P']])
            if 'DURATION' in doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['OBJECT_DESCR'][0].keys():
                d['OBJECT_DESCR_DURATION'] = doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['OBJECT_DESCR'][0]['DURATION']['#text'] + ' ' + doc['FORM_SECTION']['F02_2014']['OBJECT_CONTRACT']['OBJECT_DESCR'][0]['DURATION']['@TYPE']
        d['PROCEDURE_DATE_RECEIPT_TENDERS'] = doc['FORM_SECTION']['F02_2014']['PROCEDURE']['DATE_RECEIPT_TENDERS'] if 'DATE_RECEIPT_TENDERS' in doc['FORM_SECTION']['F02_2014']['PROCEDURE'].keys() else ''
        d['PROCEDURE_TIME_RECEIPT_TENDERS'] = doc['FORM_SECTION']['F02_2014']['PROCEDURE']['TIME_RECEIPT_TENDERS'] if 'TIME_RECEIPT_TENDERS' in doc['FORM_SECTION']['F02_2014']['PROCEDURE'].keys() else ''
        d['PROCEDURE_DURATION_TENDER_VALID'] = doc['FORM_SECTION']['F02_2014']['PROCEDURE']['DURATION_TENDER_VALID']['#text'] + ' ' + doc['FORM_SECTION']['F02_2014']['PROCEDURE']['DURATION_TENDER_VALID']['@TYPE'] if 'DURATION_TENDER_VALID' in doc['FORM_SECTION']['F02_2014']['PROCEDURE'].keys() else ''
        if 'OPENING_CONDITION' in doc['FORM_SECTION']['F02_2014']['PROCEDURE'].keys():
            d['PROCEDURE_DATE_OPENING_TENDERS'] = doc['FORM_SECTION']['F02_2014']['PROCEDURE']['OPENING_CONDITION']['DATE_OPENING_TENDERS'] if 'DATE_OPENING_TENDERS' in doc['FORM_SECTION']['F02_2014']['PROCEDURE']['OPENING_CONDITION'].keys() else ''
            d['PROCEDURE_TIME_OPENING_TENDERS'] = doc['FORM_SECTION']['F02_2014']['PROCEDURE']['OPENING_CONDITION']['TIME_OPENING_TENDERS'] if 'TIME_OPENING_TENDERS' in doc['FORM_SECTION']['F02_2014']['PROCEDURE']['OPENING_CONDITION'].keys() else ''
        d['DATE_DISPATCH_NOTICE'] = doc['FORM_SECTION']['F02_2014']['COMPLEMENTARY_INFO']['DATE_DISPATCH_NOTICE'] if 'DATE_DISPATCH_NOTICE' in doc['FORM_SECTION']['F02_2014']['COMPLEMENTARY_INFO'].keys() else ''
    
    return d

In [ ]:
#ToDo: the FORM section

def R209_CAN(doc, EN_POS):
    pass

def R209_PIN(doc, EN_POS):
    pass

def R208_CN(doc, EN_POS):
    pass

def R208_CAN(doc, EN_POS):
    pass

def R208_PIN(doc, EN_POS):
    pass

In [5]:
# the CODED_DATA section

def coded_and_translation_data(xml_file):
    d = dict()
    with open(xml_file, encoding='utf8') as f:
        doc = xmltodict.parse(f.read())['TED_EXPORT']
        
    # DOC_ID
    d['DOC_ID'] = doc['@DOC_ID']
    
    # EDITION
    d['EDITION'] = doc['@EDITION']
        
    # CODED_DATA_SECTION - REF_OJS
    d['COLL_OJ'] = doc['CODED_DATA_SECTION']['REF_OJS']['COLL_OJ']
    d['No_OJ'] = doc['CODED_DATA_SECTION']['REF_OJS']['NO_OJ']
    d['DATE_PUB'] = doc['CODED_DATA_SECTION']['REF_OJS']['DATE_PUB']
    
    # CODED_DATA_SECTION - NOTICE_DATA
    d['NO_DOC_OJS'] = doc['CODED_DATA_SECTION']['NOTICE_DATA']['NO_DOC_OJS']
    try:
        d['URI_LIST'] = doc['CODED_DATA_SECTION']['NOTICE_DATA']['URI_LIST']['URI_DOC'][EN_POS]['#text']
    except:
        d['URI_LIST'] = doc['CODED_DATA_SECTION']['NOTICE_DATA']['URI_LIST']['URI_DOC']['#text']
        
    d['LG_ORIG'] = doc['CODED_DATA_SECTION']['NOTICE_DATA']['LG_ORIG']
    d['ISO_COUNTRY'] = doc['CODED_DATA_SECTION']['NOTICE_DATA']['ISO_COUNTRY']['@VALUE']
    d['ORIGINAL_CPV_CODE'] = ';'.join(cpv['@CODE'] for cpv in doc['CODED_DATA_SECTION']['NOTICE_DATA']['ORIGINAL_CPV']) if isinstance(doc['CODED_DATA_SECTION']['NOTICE_DATA']['ORIGINAL_CPV'], list) else doc['CODED_DATA_SECTION']['NOTICE_DATA']['ORIGINAL_CPV']['@CODE']
    d['ORIGINAL_CPV'] = ';'.join(cpv['#text'] for cpv in doc['CODED_DATA_SECTION']['NOTICE_DATA']['ORIGINAL_CPV']) if isinstance(doc['CODED_DATA_SECTION']['NOTICE_DATA']['ORIGINAL_CPV'], list) else doc['CODED_DATA_SECTION']['NOTICE_DATA']['ORIGINAL_CPV']['#text']
    if 'n2016:PERFORMANCE_NUTS' in doc['CODED_DATA_SECTION']['NOTICE_DATA'].keys():
        if isinstance(doc['CODED_DATA_SECTION']['NOTICE_DATA']['n2016:PERFORMANCE_NUTS'], dict):
            d['n2016-PERFORMANCE_NUTS_CODE'] = doc['CODED_DATA_SECTION']['NOTICE_DATA']['n2016:PERFORMANCE_NUTS']['@CODE']
            d['n2016-PERFORMANCE_NUTS'] = doc['CODED_DATA_SECTION']['NOTICE_DATA']['n2016:PERFORMANCE_NUTS']['#text']
        else:
            d['n2016-PERFORMANCE_NUTS_CODE'] = ','.join([item['@CODE'] for item in doc['CODED_DATA_SECTION']['NOTICE_DATA']['n2016:PERFORMANCE_NUTS']])
            d['n2016-PERFORMANCE_NUTS'] = ','.join([item['#text'] for item in doc['CODED_DATA_SECTION']['NOTICE_DATA']['n2016:PERFORMANCE_NUTS']])
    if 'n2016:CA_CE_NUTS' in doc['CODED_DATA_SECTION']['NOTICE_DATA'].keys():
        if isinstance(doc['CODED_DATA_SECTION']['NOTICE_DATA']['n2016:CA_CE_NUTS'], dict):
            d['n2016-CA_CE_NUTS_CODE'] = doc['CODED_DATA_SECTION']['NOTICE_DATA']['n2016:CA_CE_NUTS']['@CODE']
            d['n2016-CA_CE_NUTS'] = doc['CODED_DATA_SECTION']['NOTICE_DATA']['n2016:CA_CE_NUTS']['#text']
        elif isinstance(doc['CODED_DATA_SECTION']['NOTICE_DATA']['n2016:CA_CE_NUTS'], list):
            d['n2016-CA_CE_NUTS_CODE'] = ','.join(item['@CODE'] for item in doc['CODED_DATA_SECTION']['NOTICE_DATA']['n2016:CA_CE_NUTS'])
            d['n2016-CA_CE_NUTS'] = ','.join(item['#text'] for item in doc['CODED_DATA_SECTION']['NOTICE_DATA']['n2016:CA_CE_NUTS'])
    
    # CODED_DATA_SECTION - CODIF_DATA
    d['DS_DATE_DISPATCH'] = doc['CODED_DATA_SECTION']['CODIF_DATA']['DS_DATE_DISPATCH']
    d['DT_DATE_FOR_SUBMISSION'] = doc['CODED_DATA_SECTION']['CODIF_DATA']['DT_DATE_FOR_SUBMISSION']
    d['AA_AUTHORITY_TYPE'] = doc['CODED_DATA_SECTION']['CODIF_DATA']['AA_AUTHORITY_TYPE']['#text']
    d['TD_DOCUMENT_TYPE'] = doc['CODED_DATA_SECTION']['CODIF_DATA']['TD_DOCUMENT_TYPE']['#text']
    d['NC_CONTRACT_NATURE'] = doc['CODED_DATA_SECTION']['CODIF_DATA']['NC_CONTRACT_NATURE']['#text']
    d['PR_PROC'] = doc['CODED_DATA_SECTION']['CODIF_DATA']['PR_PROC']['#text']
    d['RP_REGULATION'] = doc['CODED_DATA_SECTION']['CODIF_DATA']['RP_REGULATION']['#text']
    d['TY_TYPE_BID'] = doc['CODED_DATA_SECTION']['CODIF_DATA']['TY_TYPE_BID']['#text']
    d['AC_AWARD_CRIT'] = doc['CODED_DATA_SECTION']['CODIF_DATA']['AC_AWARD_CRIT']['#text']
    if 'MA_MAIN_ACTIVITIES' in doc['CODED_DATA_SECTION']['CODIF_DATA'].keys():
        if isinstance(doc['CODED_DATA_SECTION']['CODIF_DATA']['MA_MAIN_ACTIVITIES'], dict):
            d['MA_MAIN_ACTIVITIES'] = doc['CODED_DATA_SECTION']['CODIF_DATA']['MA_MAIN_ACTIVITIES']['#text']
        else:
            d['MA_MAIN_ACTIVITIES'] = ','.join(item['#text'] for item in doc['CODED_DATA_SECTION']['CODIF_DATA']['MA_MAIN_ACTIVITIES'])
    d['HEADING'] = doc['CODED_DATA_SECTION']['CODIF_DATA']['HEADING']
    d['INITIATOR'] = doc['CODED_DATA_SECTION']['CODIF_DATA']['INITIATOR'] if 'INITIATOR' in doc['CODED_DATA_SECTION']['CODIF_DATA'].keys() else ''
    
    # TRANSLATION_SECTION - ML_TITLES
    d['TITLE'] = doc['TRANSLATION_SECTION']['ML_TITLES']['ML_TI_DOC'][EN_POS]['TI_TEXT']['P']
    d['CITY'] = doc['TRANSLATION_SECTION']['ML_TITLES']['ML_TI_DOC'][EN_POS]['TI_CY']
    d['TOWN'] = doc['TRANSLATION_SECTION']['ML_TITLES']['ML_TI_DOC'][EN_POS]['TI_TOWN']
    
    # TRANSLATION_SECTION - ML_AA_NAMES
    try:
        d['CONTRACTING AUTHORITY'] = doc['TRANSLATION_SECTION']['ML_AA_NAMES']['AA_NAME'][EN_POS]['#text']
    except:
        d['CONTRACTING AUTHORITY'] = doc['TRANSLATION_SECTION']['ML_AA_NAMES']['AA_NAME']['#text']
        
    return d

In [13]:
# to call all parsing helper functions & parse the XML
def parse(dirName):
    xmlFilesList = list()
    for (dirpath, dirnames, filenames) in os.walk(dirName):
        xmlFilesList += [os.path.join(dirpath, file) for file in filenames]
        
    for i, file in enumerate(xmlFilesList):
        if file.endswith('.xml'):
            
            print('[info] {} ... '.format(file), end='')
            with open(file, encoding='utf8') as f:
                doc = xmltodict.parse(f.read())['TED_EXPORT']
            
            EN_POS = get_lang_pos(doc)
            VERSION = get_version(doc)
            FORM = get_form(doc)
            
            print('Version: {}  FORM: {}'.format(VERSION, FORM))
            
        
parse('FTP_Data/2019/')

[info] FTP_Data/2019/2019-01\20190102_001\000001_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190102_001\000002_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CAN
[info] FTP_Data/2019/2019-01\20190102_001\000003_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CAN
[info] FTP_Data/2019/2019-01\20190102_001\000004_2019.xml ... Version: R2.0.8.S04.E01  FORM: R2.0.8_OTH_NOT
[info] FTP_Data/2019/2019-01\20190102_001\000005_2019.xml ... Version: R2.0.8.S04.E01  FORM: R2.0.8_OTH_NOT
[info] FTP_Data/2019/2019-01\20190102_001\000006_2019.xml ... Version: R2.0.8.S04.E01  FORM: R2.0.8_OTH_NOT
[info] FTP_Data/2019/2019-01\20190102_001\000007_2019.xml ... Version: R2.0.8.S04.E01  FORM: R2.0.8_OTH_NOT
[info] FTP_Data/2019/2019-01\20190102_001\000008_2019.xml ... Version: R2.0.8.S04.E01  FORM: R2.0.8_OTH_NOT
[info] FTP_Data/2019/2019-01\20190102_001\000009_2019.xml ... Version: R2.0.8.S04.E01  FORM: R2.0.8_OTH_NOT
[info] FTP_Data/2019/2019-01\20190102_001

[info] FTP_Data/2019/2019-01\20190102_001\000091_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CAN
[info] FTP_Data/2019/2019-01\20190102_001\000092_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CAN
[info] FTP_Data/2019/2019-01\20190102_001\000093_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CAN
[info] FTP_Data/2019/2019-01\20190102_001\000094_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CAN
[info] FTP_Data/2019/2019-01\20190102_001\000095_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CAN
[info] FTP_Data/2019/2019-01\20190102_001\000096_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CAN
[info] FTP_Data/2019/2019-01\20190102_001\000097_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CAN
[info] FTP_Data/2019/2019-01\20190102_001\000098_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CAN
[info] FTP_Data/2019/2019-01\20190102_001\000099_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CAN
[info] FTP_Data/2019/2019-01\20190102_001\000100_2019.xml ... Ve

[info] FTP_Data/2019/2019-01\20190102_001\000174_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190102_001\000175_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190102_001\000176_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190102_001\000177_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190102_001\000178_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190102_001\000179_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190102_001\000180_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190102_001\000181_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190102_001\000182_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190102_001\000183_2019.xml ... Version: R2

[info] FTP_Data/2019/2019-01\20190102_001\000261_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190102_001\000262_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190102_001\000263_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190102_001\000264_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190102_001\000265_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190102_001\000266_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190102_001\000267_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190102_001\000268_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190102_001\000269_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190102_001\000270_2019.xml ... Version: R2

[info] FTP_Data/2019/2019-01\20190102_001\000344_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CAN
[info] FTP_Data/2019/2019-01\20190102_001\000345_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CAN
[info] FTP_Data/2019/2019-01\20190102_001\000346_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CAN
[info] FTP_Data/2019/2019-01\20190102_001\000347_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CAN
[info] FTP_Data/2019/2019-01\20190102_001\000348_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CAN
[info] FTP_Data/2019/2019-01\20190102_001\000349_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CAN
[info] FTP_Data/2019/2019-01\20190102_001\000350_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CAN
[info] FTP_Data/2019/2019-01\20190102_001\000351_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CAN
[info] FTP_Data/2019/2019-01\20190102_001\000352_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CAN
[info] FTP_Data/2019/2019-01\20190102_001\000353_2019.xml ... Ve

...

[info] FTP_Data/2019/2019-01\20190108_005\006062_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CAN
[info] FTP_Data/2019/2019-01\20190108_005\006063_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CAN
[info] FTP_Data/2019/2019-01\20190108_005\006064_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CAN
[info] FTP_Data/2019/2019-01\20190108_005\006065_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CAN
[info] FTP_Data/2019/2019-01\20190108_005\006066_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CAN
[info] FTP_Data/2019/2019-01\20190108_005\006067_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CAN
[info] FTP_Data/2019/2019-01\20190108_005\006068_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CAN
[info] FTP_Data/2019/2019-01\20190108_005\006069_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CAN
[info] FTP_Data/2019/2019-01\20190108_005\006070_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CAN
[info] FTP_Data/2019/2019-01\20190108_005\006071_2019.xml ... Ve

[info] FTP_Data/2019/2019-01\20190108_005\006140_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_F14_2014
[info] FTP_Data/2019/2019-01\20190108_005\006141_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_F14_2014
[info] FTP_Data/2019/2019-01\20190108_005\006142_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_F14_2014
[info] FTP_Data/2019/2019-01\20190108_005\006143_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_F14_2014
[info] FTP_Data/2019/2019-01\20190108_005\006144_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_F14_2014
[info] FTP_Data/2019/2019-01\20190108_005\006145_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_F14_2014
[info] FTP_Data/2019/2019-01\20190108_005\006146_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_F14_2014
[info] FTP_Data/2019/2019-01\20190108_005\006147_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_F14_2014
[info] FTP_Data/2019/2019-01\20190108_005\006148_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_F14_2014
[info] FTP_Data/201

[info] FTP_Data/2019/2019-01\20190108_005\006236_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190108_005\006237_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190108_005\006238_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190108_005\006239_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190108_005\006240_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190108_005\006241_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190108_005\006242_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190108_005\006243_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190108_005\006244_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190108_005\006245_2019.xml ... Version: R2

[info] FTP_Data/2019/2019-01\20190108_005\006327_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190108_005\006328_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190108_005\006329_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190108_005\006330_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190108_005\006331_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190108_005\006332_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190108_005\006333_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190108_005\006334_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190108_005\006335_2019.xml ... Version: R2.0.9.S03.E01  FORM: R2.0.9_CN
[info] FTP_Data/2019/2019-01\20190108_005\006336_2019.xml ... Version: R2

KeyboardInterrupt: 